# Model Training Pipeline

## imports

In [38]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch

## Training Job

In [25]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = "relu-bucket"

# Path to training data in S3
s3_train_path = f"s3://{bucket}/conveyor_fault_dataset.csv"

print("Dataset location:", s3_train_path)

Dataset location: s3://relu-bucket/conveyor_fault_dataset.csv


## Training Script

In [ ]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()

s3_train_path = s3_train_path

# Create estimator
estimator = PyTorch(
    entry_point="train.py",
    source_dir=".",
    role=role,
    framework_version="1.13",
    py_version="py39",
    instance_count=1,
    instance_type="ml.m4.xlarge",
    hyperparameters={
        "epochs": 20,
        "batch-size": 64,
        "hidden-size": 128,
        "num-layers": 2,
        "seq-len": 30,
        "lr": 1e-3
    },
)

# Start training
estimator.fit({"train": s3_train_path})


# Inference

In [78]:
# Since you already have an endpoint deployed, let's fix the serialization issue
# Run this code to configure your existing predictor properly:

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor
import sagemaker
import json

# Get your session
session = sagemaker.Session()

# Connect to your existing endpoint
endpoint_name = "pytorch-inference-2025-09-11-13-37-04-346"  # Your endpoint name from the output above
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=session)

# Configure JSON serialization (this is what was missing!)
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

print("Configured predictor with JSON serializers")

# Test the endpoint with proper serialization
print("\nTesting the endpoint...")

test_payload = {
    "instances": [
        {
            "Speed (rpm)": 120.0,
            "Load (kg)": 30.0,
            "Temperature (℃)": 40.0,
            "Vibration (m/s²)": 0.9,
            "Current (A)": 3.2
        },
        {
            "Speed (rpm)": 150.0,
            "Load (kg)": 45.0,
            "Temperature (℃)": 55.0,
            "Vibration (m/s²)": 1.2,
            "Current (A)": 4.1
        }
    ]
}

try:
    response = predictor.predict(test_payload)
    print("Success!")
    print("Response:", response)
    
    # Parse and display results nicely
    result = response  # Should already be parsed due to JSONDeserializer
    print("\nPrediction Results:")
    for i, pred in enumerate(result["predictions"]):
        print(f"Sample {i+1}:")
        print(f"  Predicted Class: {pred['predicted_class']}")
        print(f"  Confidence: {pred['confidence']:.4f}")
        print()
        
except Exception as e:
    print(f"Error: {e}")
    print("\nCheck CloudWatch logs for detailed error information:")
    print(f"https://console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/{endpoint_name}")

# Optional: Clean up (uncomment to delete endpoint after testing)
# print("\nCleaning up endpoint...")
# predictor.delete_endpoint()
# print("Endpoint deleted.")

Configured predictor with JSON serializers

Testing the endpoint...
Success!
Response: {'predictions': [{'predicted_class': 'belt slippage', 'confidence': 0.8073613047599792, 'all_probabilities': {'ball bearing': 0.16195383667945862, 'belt slippage': 0.8073613047599792, 'central shaft': 0.00467093288898468, 'drive motor': 0.0009125770884566009, 'idler roller fault': 0.012591534294188023, 'pulley': 0.012509861961007118}}, {'predicted_class': 'belt slippage', 'confidence': 0.47998523712158203, 'all_probabilities': {'ball bearing': 0.3270701766014099, 'belt slippage': 0.47998523712158203, 'central shaft': 0.11478602886199951, 'drive motor': 0.015154252760112286, 'idler roller fault': 0.02549790032207966, 'pulley': 0.037506427615880966}}]}

Prediction Results:
Sample 1:
  Predicted Class: belt slippage
  Confidence: 0.8074

Sample 2:
  Predicted Class: belt slippage
  Confidence: 0.4800



In [77]:
import json

payload = {
    "instances": [
        {"Speed (rpm)": 120, "Load (kg)": 300, "Temperature (℃)": 60, "Vibration (m/s²)": 1.2, "Current (A)": 10}
    ]
}


response = predictor.predict(payload)
print(response)


{'predictions': [{'predicted_class': 'drive motor', 'confidence': 0.8380087018013, 'all_probabilities': {'ball bearing': 0.001289673033170402, 'belt slippage': 0.0015837793471291661, 'central shaft': 0.1338430792093277, 'drive motor': 0.8380087018013, 'idler roller fault': 0.011230221949517727, 'pulley': 0.014044506475329399}}]}
